In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
import time
import requests
from tqdm import tqdm

# Function to download images
def download_image(image_url, folder_path, filename):
    response = requests.get(image_url, stream=True)
    if response.status_code == 200:
        with open(os.path.join(folder_path, filename), 'wb') as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)

cService = webdriver.ChromeService(executable_path=r"C:\Users\user\Documents\chromedriver-win64\chromedriver.exe")
driver = webdriver.Chrome(service=cService)
driver.get("https://www.pupvote.com/us/leaderboard?type=cats")
print("Loaded")
# Wait for JavaScript content to load
time.sleep(3)

Loaded


In [2]:
from IPython.display import clear_output

kittens = driver.find_elements(By.CLASS_NAME, "Leaderboard_leaderboardItem__VQz_J")  # Adjust class name

while len(kittens) < 2500:
    try:
        show_more = driver.find_element(By.CLASS_NAME, "Leaderboard_leaderboardShowmore__369gF").find_element(By.TAG_NAME, "button")
    except:
        break
    show_more.click()
    kittens = driver.find_elements(By.CLASS_NAME, "Leaderboard_leaderboardItem__VQz_J")
    clear_output()
    print(len(kittens))
    time.sleep(2.5)
clear_output()
print(len(kittens))

kitten_urls = []
for idx, kitten in enumerate(kittens):
    kitten_urls.append((kitten.get_attribute("href"), idx))

# Close the browser
driver.quit()

2040


In [ ]:
from concurrent.futures import ThreadPoolExecutor

def process_kitten(kitten_data):
    cService = webdriver.ChromeService(executable_path=r"C:\Users\user\Documents\chromedriver-win64\chromedriver.exe")
    driver = webdriver.Chrome(service=cService)
    kitten_url = kitten_data[0]
    kitten_idx = kitten_data[1]
    driver.get(kitten_url)
    time.sleep(5)

    try:
        # Step 1: Extract kitten name
        kitten_name = str(kitten_idx+1) + driver.find_element(By.CLASS_NAME, "pet-top").find_element(By.TAG_NAME, "h1").text.strip() 
        print(f"Processing {kitten_name}...")

        # Step 2: Find all image URLs
        while True:
            try:
                driver.find_element(By.CLASS_NAME, "Photos_moreImages__10L1r").click()
                time.sleep(1)
            except:
                break

        if len(driver.find_elements(By.CLASS_NAME, "Photos_image__2se20")) < 1:
            raise Exception("There are less than 10 pictures")
            
        image_urls = [
            driver.find_element(By.CLASS_NAME, "pet-image").find_element(By.TAG_NAME, "img").get_attribute('src')
            ]
        mini_images = driver.find_elements(By.CLASS_NAME, "Photos_image__2se20")
        
        for mini_img in mini_images:
            mini_img.click()
            time.sleep(1)
            block = driver.find_element(By.CLASS_NAME, "PhotoPreview_image__1face")
            image = block.find_element(By.TAG_NAME, "img")
            image_urls.append(image.get_attribute("src"))
            
            btn = driver.find_element(By.CLASS_NAME, "PhotoPreview_closeButton__29-CE")
            btn.click()
            time.sleep(1.5)

        # Step 3: Create a folder for the kitten's images
        folder_path = os.path.join("kittens", kitten_name)
        os.makedirs(folder_path, exist_ok=True)

        # Step 4: Download each image
        print(f"Downloading images for {kitten_name}...")
        for idx, image_url in enumerate(tqdm(image_urls)):
            filename = f"{kitten_name}_{idx + 1}.jpg"
            download_image(image_url, folder_path, filename)

        print(f"All images for {kitten_name} have been downloaded to {folder_path}.")
    finally:
        # Close the browser for this thread
        clear_output()
        driver.quit()

# Main function for parallel processing
def main(kitten_urls):
    with ThreadPoolExecutor(max_workers=6) as executor:  # Adjust the number of workers as needed
        executor.map(process_kitten, kitten_urls)

main(kitten_urls)
